<a href="https://colab.research.google.com/github/KevinGomez26/Pain_classification/blob/main/EEGNet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install braindecode
import braindecode
from huggingface_hub import hf_hub_download
from braindecode.models import EEGNetv4

import pickle
import torch
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
path_kwargs = hf_hub_download(
    repo_id='PierreGtch/EEGNetv4',
    filename='EEGNetv4_Lee2019_MI/kwargs.pkl',
)
path_params = hf_hub_download(
    repo_id='PierreGtch/EEGNetv4',
    filename='EEGNetv4_Lee2019_MI/model-params.pkl',
)
with open(path_kwargs, 'rb') as f:
    kwargs = pickle.load(f)
module_cls = kwargs['module_cls']
module_kwargs = kwargs['module_kwargs']

torch_module = module_cls(**module_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


kwargs.pkl:   0%|          | 0.00/164 [00:00<?, ?B/s]

model-params.pkl:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'in_chans' is depreciated. Use 'n_chans' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'n_classes' is depreciated. Use 'n_outputs' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(


In [ ]:
def EEGNet_feature_extractor(raw, window_size, mode_type):

  if mode_type == 'complete':
    duration_in_seconds =  raw.n_times / raw.info['sfreq']
    time_start = 0
    time_end = time_start + window_size
    instance_limit = 30
  elif mode_type == 'open_eyes':
    duration_in_seconds =  (raw.n_times / raw.info['sfreq']) / 2
    time_start = 0
    time_end = time_start + window_size
    instance_limit = 15
  elif mode_type == 'closed_eyes':
    duration_in_seconds =  raw.n_times / raw.info['sfreq']
    time_start = duration_in_seconds / 2
    time_end = time_start + window_size
    instance_limit = 15

  eeg_features = []
  while time_end <= duration_in_seconds:

    raw_segment = raw.copy().crop(tmin=time_start, tmax=time_end)
    eeg = raw_segment.get_data()
    eeg = eeg.reshape(1, eeg.shape[0], eeg.shape[1])
    eeg = torch.tensor(eeg, dtype=torch.float64)

    eegnet = EEGNetv4(in_chans=eeg.shape[1], n_classes=2, input_window_samples=eeg.shape[2])
    eegnet.double()
    eegnet_layers = list(eegnet.children())
    eegnet_feature = torch.nn.Sequential(*eegnet_layers[:-1])

    with torch.no_grad():
        features = eegnet_feature(eeg)
        features_flatten = torch.nn.Flatten()(features)
        features_dense = torch.nn.Linear(in_features=features_flatten.size(1), out_features=768, dtype=torch.double)(features_flatten)

    eeg_features.append(features_dense.numpy())
    time_end +=window_size
    time_start +=window_size

  if len(eeg_features) >= instance_limit:
    eeg_features.pop()

  eeg_features = np.array(eeg_features)
  eeg_features = np.squeeze(eeg_features, axis=1)

  return eeg_features